# データ
次の辞書ファイルを作成する

## 姓名辞書
- oss.json
オープンソースの辞書から作成した，漢字・アルファベットとカタカナのペアデータ
- seimei.json
ダウンロードしたWebデータ（Wikipedia）から作成した漢字・アルファベットとカタカナのペアのデータ

## 単漢字辞書
- tankanji.json
オープンソースの辞書から作成した，単漢字とカタカナのペアデータ

## 異体字辞書
- itaiji.json
オープンソースの辞書から作成した，異体字の漢字のペアデータ




# データを準備する

辞書用及び。AI用のデータを，オープンソースのデータからダウンロードし，整形する。
使用するデータソースは次のとおり      

## データソース一覧

### オープンソース辞書データ
- [canna](https://osdn.dl.osdn.net/canna/)
- [freewnn](https://osdn.dl.osdn.net/freewnn/)
- [ipadic](https://taku910.github.io/mecab/)
- [kakasi](http://kakasi.namazu.org/)
- [mj](https://moji.or.jp/)
- [mozc](https://github.com/google/mozc/)
- [neologd](https://github.com/neologd/mecab-ipadic-neologd/)
- [skk](https://github.com/skk-dev/dict/)

### Webデータ
- [wikipedia](https://dumps.wikimedia.org/)

## データ形式について
データは，単語辞書，単漢字辞書，異体字辞書をそれぞれのデータソースから作成する。
作成するデータフォーマットはJSON形式であり，項目については次のとおり。

### 姓名辞書
単語漢字をキーとし，その読みのカタカナをキーとするハッシュテーブルをバリューとして追加する。またカタカナにはそれぞれdicsというキーで，漢字とカタカナのペアが存在する，データソースの一覧を保持している。
```json
{ 
    "単語漢字１" :{
        "カタカナ１":{
            "dics":[
                "freewnn",
                "canna"
            ]
        },
        "カタカナ２":{
            "dics":{
                "canna"
            }
        }
    },
    "単語漢字２":{
        ..
    }
..

```

### 単漢字辞書
単漢字をキーとし，その読みのカタカナをキーとするハッシュテーブルをバリューとして追加する。またカタカナにはそれぞれdicsというキーで，漢字とカタカナのペアが存在する，データソースの一覧を保持している。
```json
{ 
    "単漢字１" :{
        "カタカナ１":{
            "dics":[
                "freewnn",
                "canna"
            ]
        },
        "カタカナ２":{
            "dics":{
                "canna"
            }
        }
    },
    "単漢字２":{
        ..
    }
..

```


### 異体字辞書
単漢字をキーとし，その異体字である単漢字をキーとするハッシュテーブルをバリューとして追加する。また異体字である単漢字にはそれぞれdicsというキーで，漢字とカタカナのペアが存在する，データソースの一覧を保持している。

```json
{ 
    "単漢字１" :{
        "異体字１":{
            "dics":[
                "kakasi",
                "mozc"
            ]
        },
        "異体字２":{
            "dics":{
                "skk"
            }
        }
    },
    "単漢字２":{
        ..
    }
..

```

## 事前準備
pythonで必要なライブラリ群をインストールしておく

In [1]:
# warning off
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install -r ../requirements.txt;



In [3]:
!pip install wget

# データをダウンロードし，漢字とカタカナを抜き出す
あらかじめ，nkfコマンドをインストールしておく必要がある。

## canna
cannnaからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [4]:

%cd canna
!wget -nc -q "https://osdn.dl.osdn.net/canna/9565/Canna37p3.tar.bz2" --no-check-certificate;
!bunzip2 Canna37p3.tar.bz2;
!tar xfp Canna37p3.tar;
!bzip2 Canna37p3.tar;



/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/canna


In [5]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_canna.py --indir Canna37p3/dic/ideo/pubdic --outfile extract.txt
!python words.py --indir "Canna37p3/dic/ideo/words" --outfile words.txt
!cat extract.txt words.txt > dict.txt


{
  "indir": "Canna37p3/dic/ideo/pubdic",
  "outfile": "extract.txt"
}
{
  "indir": "Canna37p3/dic/ideo/words",
  "outfile": "words.txt"
}


In [6]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt


{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [7]:
# cannaのダウンロードデータを削除する
!rm -rf Canna37p3
%cd ..


/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict


## freewnn
freewnnからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [8]:
%cd freewnn
!wget -nc -q https://osdn.dl.osdn.net/freewnn/63271/FreeWnn-1.1.1-a023.tar.gz --no-check-certificate
!tar zxfp FreeWnn-1.1.1-a023.tar.gz



/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/freewnn


In [9]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_freewnn.py --infile FreeWnn-1.1.1-a023/PubdicPlus/pubdic.p --outfile dict.txt
!rm -rf FreeWnn-1.1.1-a023


{
  "infile": "FreeWnn-1.1.1-a023/PubdicPlus/pubdic.p",
  "outfile": "dict.txt"
}


In [10]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt


{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [11]:

# freewnnのダウンロードデータを削除する
!rm -rf FreeWnn-1.1.1-a023
%cd ..


/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict



## ipadic
ipadicからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。

In [12]:
%cd ipadic
!wget -nc -q --no-check-certificate  "https://ja.osdn.net/frs/g_redir.php?m=ipconnect&f=mecab%2Fmecab-ipadic%2F2.7.0-20070801%2Fmecab-ipadic-2.7.0-20070801.tar.gz" -O mecab-ipadic-2.7.0-20070801.tar.gz
!tar zxfp mecab-ipadic-2.7.0-20070801.tar.gz



/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/ipadic


In [13]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_ipadic.py --indir="mecab-ipadic-2.7.0-20070801/" --outfile="dict.txt"



{
  "indir": "mecab-ipadic-2.7.0-20070801/",
  "outfile": "dict.txt"
}
mecab-ipadic-2.7.0-20070801/Adj.csv
mecab-ipadic-2.7.0-20070801/Adnominal.csv
mecab-ipadic-2.7.0-20070801/Adverb.csv
mecab-ipadic-2.7.0-20070801/Auxil.csv
mecab-ipadic-2.7.0-20070801/Conjunction.csv
mecab-ipadic-2.7.0-20070801/Filler.csv
mecab-ipadic-2.7.0-20070801/Interjection.csv
mecab-ipadic-2.7.0-20070801/Noun.adjv.csv
mecab-ipadic-2.7.0-20070801/Noun.adverbal.csv
mecab-ipadic-2.7.0-20070801/Noun.csv
mecab-ipadic-2.7.0-20070801/Noun.demonst.csv
mecab-ipadic-2.7.0-20070801/Noun.nai.csv
mecab-ipadic-2.7.0-20070801/Noun.name.csv
mecab-ipadic-2.7.0-20070801/Noun.number.csv
mecab-ipadic-2.7.0-20070801/Noun.org.csv
mecab-ipadic-2.7.0-20070801/Noun.others.csv
mecab-ipadic-2.7.0-20070801/Noun.place.csv
mecab-ipadic-2.7.0-20070801/Noun.proper.csv
mecab-ipadic-2.7.0-20070801/Noun.verbal.csv
mecab-ipadic-2.7.0-20070801/Others.csv
mecab-ipadic-2.7.0-20070801/Postp-col.csv
mecab-ipadic-2.7.0-20070801/Postp.csv
mecab-ipadic-2

In [14]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt



{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [15]:
# ダウンロードデータを削除する
!rm -rf mecab-ipadic-2.7.0-20070801
%cd ..


/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict


## kakasi
kakasiからは，単語辞書と単漢字辞書，異体字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。

In [16]:
%cd kakasi
!wget -nc -q http://kakasi.namazu.org/stable/kakasi-2.3.6.tar.gz --no-check-certificate
!tar zxfp kakasi-2.3.6.tar.gz


/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/kakasi


In [17]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python dict_kakasi.py --infile="kakasi-2.3.6/kakasidict" --outfile="dict.txt"



In [18]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt

{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [19]:
# 異体字辞書を作成する
!python  itaiji_kakasi.py --infile="kakasi-2.3.6/itaijidict" --outfile="itaiji.txt"

In [20]:
# ダウンロードデータを削除する
!rm -rf kakasi-2.3.6
%cd ..

/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict


## mj
mjからは単漢字辞書を作成する。データはExcel形式で提供されているので，Excel形式をCSV形式へ変換する[xlsx2csv](https://github.com/dilshod/xlsx2csv)を利用する。

In [21]:
%cd mj
!wget -nc -q https://moji.or.jp/wp-content/uploads/2024/01/mji.00602.xlsx


/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/mj


In [22]:
# excel形式からcsvへ変換する
!xlsx2csv mji.00602.xlsx mj.csv

In [23]:
# 単漢字辞書を作成する。
!python extract_xlsx.py --infile mj.csv --outfile tankanji.txt

{
  "infile": "mj.csv",
  "outfile": "tankanji.txt"
}
cnt=105175
len=51673


In [24]:
# ダウンロードしたファイルと不要ファイルを削除する
!rm -rf *.csv
%cd ..

/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict


## mozc
mozcからは，単語辞書と単漢字辞書，異体字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。



In [25]:
%cd mozc
!wget -nc -q "https://github.com/google/mozc/archive/refs/tags/2.29.5268.102.tar.gz" --no-check-certificate
!tar xfpz 2.29.5268.102.tar.gz

/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/mozc


In [26]:
# 単語辞書を作成する
!python extract_mozc.py --indir="mozc-2.29.5268.102/src/data/dictionary_oss/" --outfile="dict1.txt"
!python extract_oss.py --indir="mozc-2.29.5268.102/src/data/oss/" --outfile="dict2.txt"
!cat dict1.txt dict2.txt > dict.txt
!rm -rf dict1.txt dict2.txt


/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/mozc/extract_mozc.py:34: SyntaxWarning: invalid escape sequence '\p'
  if regex.match('^\p{Katakana}+$', k) is None:
{
  "indir": "mozc-2.29.5268.102/src/data/dictionary_oss/",
  "outfile": "dict1.txt"
}
mozc-2.29.5268.102/src/data/dictionary_oss/dictionary09.txt
mozc-2.29.5268.102/src/data/dictionary_oss/dictionary08.txt
mozc-2.29.5268.102/src/data/dictionary_oss/dictionary00.txt
mozc-2.29.5268.102/src/data/dictionary_oss/dictionary01.txt
mozc-2.29.5268.102/src/data/dictionary_oss/dictionary03.txt
mozc-2.29.5268.102/src/data/dictionary_oss/dictionary02.txt
mozc-2.29.5268.102/src/data/dictionary_oss/dictionary06.txt
mozc-2.29.5268.102/src/data/dictionary_oss/dictionary07.txt
mozc-2.29.5268.102/src/data/dictionary_oss/dictionary05.txt
mozc-2.29.5268.102/src/data/dictionary_oss/dictionary04.txt
/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/mozc/extract_oss.py:34: SyntaxWarning: invalid escape sequence '\p'
  if

In [27]:
# 単漢字辞書を作成する
!python extract_single.py --indir="mozc-2.29.5268.102/src/data/single_kanji/" --outfile="tankanji.txt"


{
  "indir": "mozc-2.29.5268.102/src/data/single_kanji/",
  "outfile": "tankanji.txt"
}
mozc-2.29.5268.102/src/data/single_kanji//single_kanji.tsv


In [28]:
# 異体字辞書を作成する 
!python extract_variant.py --indir="mozc-2.29.5268.102/src/data/single_kanji/" --outfile="itaiji.txt"

{
  "indir": "mozc-2.29.5268.102/src/data/single_kanji/",
  "outfile": "itaiji.txt"
}
mozc-2.29.5268.102/src/data/single_kanji//variant_rule.txt


In [29]:
# 不要ファイルを削除する
!rm -rf mozc-2.29.5268.102
%cd ..


/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict


## neologd
ipadicからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [30]:
%cd neologd
!wget -nc -q https://github.com/neologd/mecab-ipadic-neologd/archive/refs/tags/v0.0.7.tar.gz --no-check-certificate
!tar zxf v0.0.7.tar.gz
!xz --decompress  mecab-ipadic-neologd-0.0.7/seed/*xz

/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/neologd


In [31]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_neologd.py --indir="mecab-ipadic-neologd-0.0.7/seed" --outfile="dict.txt"



{
  "indir": "mecab-ipadic-neologd-0.0.7/seed",
  "outfile": "dict.txt"
}
mecab-ipadic-neologd-0.0.7/seed/mecab-user-dict-seed.20200820.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-adjective-exp-dict-seed.20151126.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-adjective-std-dict-seed.20151126.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-adjective-verb-dict-seed.20160324.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-adverb-dict-seed.20150623.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-common-noun-ortho-variant-dict-seed.20170228.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-date-time-infreq-dict-seed.20190415.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-ill-formed-words-dict-seed.20170127.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-interjection-dict-seed.20170216.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-noun-sahen-conn-ortho-variant-dict-seed.20160323.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-proper-noun-ortho-variant-dict-seed.20161110.csv
mecab-ipadic-neologd-0.0.7/seed/neologd-quantity-in

In [32]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt




{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [33]:
# 不要ファイルを削除する

!rm -rf mecab-ipadic-neologd-0.0.7
%cd ..

/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict


## skk
単語辞書と異体字辞書，単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [34]:
%cd skk
!wget -nc -q "https://github.com/skk-dev/dict/raw/master/SKK-JISYO.jinmei" --no-check-certificate
!wget -nc -q "https://github.com/skk-dev/dict/raw/master/SKK-JISYO.fullname" --no-check-certificate
!wget -nc -q https://github.com/skk-dev/dict/raw/master/SKK-JISYO.itaiji --no-check-certificate

/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/skk


In [35]:
# 姓名辞書を作成する
!python jinmei.py --infile SKK-JISYO.jinmei --outfile skk_jinmei.txt
!python fullname.py --infile SKK-JISYO.fullname --outfile skk_fullname.txt
!cat skk_jinmei.txt skk_fullname.txt > dict.txt


{
  "infile": "SKK-JISYO.jinmei",
  "outfile": "skk_jinmei.txt"
}
{
  "infile": "SKK-JISYO.fullname",
  "outfile": "skk_fullname.txt"
}


In [36]:
# 単漢字辞書を作成する
!python ../tankan.py --infile dict.txt --outfile tankanji.txt



{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [37]:
# 異体字辞書を作成する
!python itaiji_skk.py --infile SKK-JISYO.itaiji  --outfile itaiji.txt


{
  "infile": "SKK-JISYO.itaiji",
  "outfile": "itaiji.txt"
}


In [38]:
# 不要ファイルを削除する

%cd ..

/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict


## wikipedia
wikipediaからは，概要部分を抜き出して，文頭に　「漢字（読み）・・」　と記載されている場合に，漢字と読み仮名のペアを抜き出す。多少間違えている可能性があるので注意。


Wikipediaの構造解析から，Abstract部分に着目する

'''アルマ・マリア・マーラー＝ヴェルフェル'''（'''Alma Maria Mahler-Werfel''', [[1879年]][[8月31日]] – [[1964年]][[12月11日]]）は、[[オーストリア]]の作曲家[[グス
タフ・マーラー]]の妻。結婚前の姓はシンドラー（Schindler）。自身も作曲を行い、16>の歌曲が今日に残されている。華麗な男性遍歴で知られる。 


こんな形で入っている行をまずは抜き出す。     
- 見出しの最初に，'''人名'''
- ()の中に'''読み'''が入っている
- ()の中に[[yyyy年]]
- ()の中に[[mm月dd日]]


In [39]:
%cd wikipedia
!wget -nc -q https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2 --no-check-certificate



/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/wikipedia


In [40]:
# Wikipediaから概要だけを抜き出す
!python extract_wiki.py --infile="jawiki-latest-pages-articles.xml.bz2" --outfile="abstract.txt"



/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/wikipedia/extract_wiki.py:36: SyntaxWarning: invalid escape sequence '\['
  if re.search('\[\[\d+年\]\]',s) is None:
/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/wikipedia/extract_wiki.py:39: SyntaxWarning: invalid escape sequence '\['
  if re.search('\[\[\d+月\d+日\]\]',s) is None:
{
  "infile": "jawiki-latest-pages-articles.xml.bz2",
  "outfile": "abstract.txt"
}
i=0
i=1000000
i=2000000
i=3000000
i=4000000
i=5000000
i=6000000
i=7000000
i=8000000
i=9000000
i=10000000
i=11000000
i=12000000
i=13000000
i=14000000
i=15000000
i=16000000
i=17000000
i=18000000
i=19000000
i=20000000
i=21000000
i=22000000
i=23000000
i=24000000
i=25000000
i=26000000
i=27000000
i=28000000
i=29000000
i=30000000
i=31000000
i=32000000
i=33000000
i=34000000
i=35000000
i=36000000
i=37000000
i=38000000
i=39000000
i=40000000
i=41000000
i=42000000
i=43000000
i=44000000
i=45000000
i=46000000
i=47000000
i=48000000
i=49000000
i=50000000
i=51000000


In [44]:
# 漢字とカナのペアとなるものと推測されるものを抜き出す。
!python extract_name.py --outfile="abstract_name.txt" --infile="abstract.txt";


{
  "outfile": "abstract_name.txt",
  "infile": "abstract.txt"
}
i=0
i=10000
i=20000
i=30000
i=40000
i=50000
i=60000
i=70000
i=80000
i=90000
i=100000
i=110000
i=120000
i=130000
i=140000
i=150000
i=160000
i=170000
i=180000
i=190000
i=200000
i=210000
i=220000
i=230000


In [45]:
# extract_name.pyによってWikipediaのダンプファイルから，概要部分が抜き出されたものに対して，漢字・アルファベットとカタカナのペアのうち，正しいと推測されるものを選択する。
!python select_name.py --infile abstract_name.txt --outfile wikiname.txt --ngfile ng.txt;


{
  "infile": "abstract_name.txt",
  "outfile": "wikiname.txt",
  "ngfile": "ng.txt"
}
i=0
i=10000
i=20000
i=30000
i=40000
i=50000
i=60000
i=70000
i=80000
i=90000
i=100000
i=110000
i=120000
i=130000
i=140000
i=150000
i=160000
i=170000
i=180000
i=190000
i=200000


In [46]:
# select_name.pyによってWikipediaのダンプファイルから作成した漢字・アルファベットとカタカナのペアを単語単位に分割する
!python dict_name.py --infile wikiname.txt --outfile dict.txt

{
  "infile": "wikiname.txt",
  "outfile": "dict.txt"
}


In [49]:
# 不要ファイル削除
%cd ..


/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model


# ダウンロードしたデータから辞書を作成する

## オープンソース辞書データ
オープンソースの辞書から抜き出したデータで，oss.json,tankanji.json, itaiji.jsonを作成する


In [50]:
%cd dict

/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict


In [51]:
# oss.json
!rm -rf oss.json
!python merge.py --infile canna/dict.txt --jsonfile oss.json --dicname canna;
!python merge.py --infile freewnn/dict.txt --jsonfile oss.json --dicname freewnn;
!python merge.py --infile ipadic/dict.txt --jsonfile oss.json --dicname ipadic;
!python merge.py --infile kakasi/dict.txt --jsonfile oss.json --dicname kakasi;
!python merge.py --infile mozc/dict.txt --jsonfile oss.json --dicname mozc;
!python merge.py --infile neologd/dict.txt --jsonfile oss.json --dicname neologd;
!python merge.py --infile skk/dict.txt --jsonfile oss.json --dicname skk;

{
  "infile": "canna/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "canna"
}
{
  "infile": "freewnn/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "freewnn"
}
{
  "infile": "ipadic/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "ipadic"
}
{
  "infile": "kakasi/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "kakasi"
}
{
  "infile": "mozc/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "mozc"
}
{
  "infile": "neologd/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "neologd"
}
{
  "infile": "skk/dict.txt",
  "jsonfile": "oss.json",
  "dicname": "skk"
}


In [52]:
# tankanji.json
!rm -rf tankanji.json

!python merge.py --infile canna/tankanji.txt --jsonfile tankanji.json --dicname canna;
!python merge.py --infile freewnn/tankanji.txt --jsonfile tankanji.json --dicname freewnn;
!python merge.py --infile ipadic/tankanji.txt --jsonfile tankanji.json --dicname ipadic;
!python merge.py --infile kakasi/tankanji.txt --jsonfile tankanji.json --dicname kakasi;
!python merge.py --infile mozc/tankanji.txt --jsonfile tankanji.json --dicname mozc;
!python merge.py --infile neologd/tankanji.txt --jsonfile tankanji.json --dicname neologd;
!python merge.py --infile skk/tankanji.txt --jsonfile tankanji.json --dicname skk;
!python merge.py --infile mj/tankanji.txt --jsonfile tankanji.json --dicname mj;

{
  "infile": "canna/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "canna"
}
{
  "infile": "freewnn/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "freewnn"
}
{
  "infile": "ipadic/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "ipadic"
}
{
  "infile": "kakasi/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "kakasi"
}
{
  "infile": "mozc/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "mozc"
}
{
  "infile": "neologd/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "neologd"
}
{
  "infile": "skk/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "skk"
}
{
  "infile": "mj/tankanji.txt",
  "jsonfile": "tankanji.json",
  "dicname": "mj"
}


In [53]:
# itaiji.json
!rm -rf itaiji.json
!python merge.py --infile kakasi/itaiji.txt --jsonfile itaiji.json --dicname kakasi
!python merge.py --infile mozc/itaiji.txt --jsonfile itaiji.json --dicname mozc
!python merge.py --infile skk/itaiji.txt --jsonfile itaiji.json --dicname skk


{
  "infile": "kakasi/itaiji.txt",
  "jsonfile": "itaiji.json",
  "dicname": "kakasi"
}
{
  "infile": "mozc/itaiji.txt",
  "jsonfile": "itaiji.json",
  "dicname": "mozc"
}
{
  "infile": "skk/itaiji.txt",
  "jsonfile": "itaiji.json",
  "dicname": "skk"
}


## Webデータ
wikipediaから抜き出したデータから，姓名の漢字とカナのペアで，seimei.jsonを作成する


In [54]:
%cd wikipedia
# オープンソースの辞書のデータに入っていない漢字とカナのペアを抜き出す
!python check_dict.py --infile dict.txt --outfile dict_add.txt --oss ../oss.json;
%cd ..

/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict/wikipedia
{
  "infile": "dict.txt",
  "outfile": "dict_add.txt",
  "oss": "../oss.json"
}
total=112923,cnt=41847
/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/dict


In [55]:
!rm -rf seimei.json
!python merge.py --infile wikipedia/dict.txt --jsonfile seimei.json --dicname wikipedia


{
  "infile": "wikipedia/dict.txt",
  "jsonfile": "seimei.json",
  "dicname": "wikipedia"
}


## その他
その他，氏名漢字カナ突合モデルに必要な空ファイルを作成する


In [56]:
import os

with open("crawl.json",'w',encoding='utf-8') as f:
    f.write("{}")

with open("statistics.json",'w',encoding='utf-8') as f:
    f.write("{}")
